In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [18]:
best_model = grid_search.best_estimator_


In [19]:
import joblib

# After training, save your model to disk
joblib.dump(best_model, 'best_model.pkl')


['best_model.pkl']

In [20]:
import joblib

best_model = joblib.load('best_model.pkl')
print("Loaded best_model successfully.")


Loaded best_model successfully.


In [56]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss

# Load datasets
regular_season_results = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv")
tourney_seeds = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv")
sample_submission = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv")

# Check a sample of each
print("Regular Season Results (sample):")
print(regular_season_results.head())

print("\nTournament Seeds (sample):")
print(tourney_seeds.head())

print("\nSample Submission Stage2 (sample):")
print(sample_submission.head())


Regular Season Results (sample):
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  WFGM  WFGA  \
0    2003      10     1104      68     1328      62    N      0    27    58   
1    2003      10     1272      70     1393      63    N      0    26    62   
2    2003      11     1266      73     1437      61    N      0    24    58   
3    2003      11     1296      56     1457      50    N      0    18    38   
4    2003      11     1400      77     1208      71    N      0    30    61   

   ...  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  
0  ...     10    16    22   10   22     8   18     9     2   20  
1  ...     24     9    20   20   25     7   12     8     6   16  
2  ...     26    14    23   31   22     9   12     2     5   23  
3  ...     22     8    15   17   20     9   19     4     3   23  
4  ...     16    17    27   21   15    12   10     7     1   14  

[5 rows x 34 columns]

Tournament Seeds (sample):
   Season Seed  TeamID
0    1985  W01    1207

In [57]:
# Aggregate winning data
wins = regular_season_results.groupby("WTeamID").agg(
    Wins=('WTeamID', 'count'),
    PointsScored_W=('WScore', 'sum'),
    PointsAllowed_W=('LScore', 'sum')
).reset_index().rename(columns={"WTeamID": "TeamID"})

# Aggregate losing data
losses = regular_season_results.groupby("LTeamID").agg(
    Losses=('LTeamID', 'count'),
    PointsScored_L=('LScore', 'sum'),
    PointsAllowed_L=('WScore', 'sum')
).reset_index().rename(columns={"LTeamID": "TeamID"})

# Merge wins and losses
team_stats = pd.merge(wins, losses, on="TeamID", how="outer").fillna(0)

# Total games played
team_stats["TotalGames"] = team_stats["Wins"] + team_stats["Losses"]

# Win rate and point differential
team_stats["WinRate"] = team_stats["Wins"] / team_stats["TotalGames"]
team_stats["PointDifferential"] = (
    (team_stats["PointsScored_W"] - team_stats["PointsAllowed_W"]) +
    (team_stats["PointsScored_L"] - team_stats["PointsAllowed_L"])
) / team_stats["TotalGames"]

print("Team Stats (sample):")
print(team_stats.head())


Team Stats (sample):
   TeamID  Wins  PointsScored_W  PointsAllowed_W  Losses  PointsScored_L  \
0    1101   151           11563             9864     168           10689   
1    1102   286           20057            16402     382           22534   
2    1103   466           35341            29525     243           15954   
3    1104   441           34383            28339     279           18493   
4    1105   199           14531            12547     427           26019   

   PointsAllowed_L  TotalGames   WinRate  PointDifferential  
0            12848         319  0.473354          -1.442006  
1            27534         668  0.428144          -2.013473  
2            18233         709  0.657264           4.988717  
3            21083         720  0.612500           4.797222  
4            32191         626  0.317891          -6.690096  


In [58]:
# Compute home win rate per team from regular season results (WLoc = 'H' for win at home)
home_adv = regular_season_results.groupby("WTeamID").apply(
    lambda x: (x['WLoc'] == 'H').sum() / len(x)
).reset_index(name="HomeWinRate").rename(columns={"WTeamID": "TeamID"})

print("Home Court Advantage (sample):")
print(home_adv.head())


Home Court Advantage (sample):
   TeamID  HomeWinRate
0    1101     0.509934
1    1102     0.674825
2    1103     0.585837
3    1104     0.646259
4    1105     0.603015


<ipython-input-58-14d0d8ebb27d>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  home_adv = regular_season_results.groupby("WTeamID").apply(


In [59]:
# Define a function to arrange games so that the lower TeamID is first
def process_game(row):
    # Determine lower and higher team IDs
    team_low = min(row['WTeamID'], row['LTeamID'])
    team_high = max(row['WTeamID'], row['LTeamID'])
    # If the winning team is the lower ID, label 1; otherwise 0.
    outcome = 1 if row['WTeamID'] == team_low else 0
    return pd.Series([row['Season'], team_low, team_high, outcome], 
                     index=['Season', 'Team1_Game', 'Team2_Game', 'Outcome'])

# Apply processing to all regular season games
training_games = regular_season_results.apply(process_game, axis=1)

# Merge team-level features for the matchup (Team1)
training = pd.merge(training_games, team_stats[['TeamID', 'WinRate', 'PointDifferential']], 
                    left_on='Team1_Game', right_on='TeamID', how='left') \
           .rename(columns={'WinRate': 'WinRate_Team1', 'PointDifferential': 'PointDifferential_Team1'}) \
           .drop(columns=["TeamID"])

# Merge team-level features for Team2
training = pd.merge(training, team_stats[['TeamID', 'WinRate', 'PointDifferential']], 
                    left_on='Team2_Game', right_on='TeamID', how='left') \
           .rename(columns={'WinRate': 'WinRate_Team2', 'PointDifferential': 'PointDifferential_Team2'}) \
           .drop(columns=["TeamID"])

# Merge home court advantage metrics for Team1 and Team2
training = pd.merge(training, home_adv, left_on='Team1_Game', right_on='TeamID', how='left') \
           .rename(columns={'HomeWinRate': 'HomeWinRate_Team1'}).drop(columns=["TeamID"])
training = pd.merge(training, home_adv, left_on='Team2_Game', right_on='TeamID', how='left') \
           .rename(columns={'HomeWinRate': 'HomeWinRate_Team2'}).drop(columns=["TeamID"])

# Compute feature differences
training['WinRateDifference'] = training['WinRate_Team1'] - training['WinRate_Team2']
training['PointDifferentialDifference'] = training['PointDifferential_Team1'] - training['PointDifferential_Team2']
training['HomeWinRateDifference'] = training['HomeWinRate_Team1'] - training['HomeWinRate_Team2']

# Final training features and label
features = ['WinRateDifference', 'PointDifferentialDifference', 'HomeWinRateDifference']
X = training[features]
y = training['Outcome']

print("Training features (sample):")
print(X.head())
print("Training labels (sample):")
print(y.head())


Training features (sample):
   WinRateDifference  PointDifferentialDifference  HomeWinRateDifference
0          -0.001040                    -0.056092              -0.006615
1           0.061367                     2.381743              -0.028665
2          -0.050592                    -1.899015               0.117355
3          -0.251067                    -7.526939               0.059924
4          -0.152141                    -7.058126               0.061065
Training labels (sample):
0    1
1    1
2    1
3    1
4    0
Name: Outcome, dtype: int64


In [62]:
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='binary_logloss',
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=True)]
)


[LightGBM] [Info] Number of positive: 46322, number of negative: 48437
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 94759, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.488840 -> initscore=-0.044647
[LightGBM] [Info] Start training from score -0.044647
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[546]	valid_0's binary_logloss: 0.607817


LGBMClassifier(learning_rate=0.01, n_estimators=1000, random_state=42)

In [63]:
from sklearn.metrics import brier_score_loss

# Get predicted probabilities for the positive class on the validation set
val_preds = lgb_model.predict_proba(X_val)[:, 1]
val_brier = brier_score_loss(y_val, val_preds)
print("Validation Brier Score:", val_brier)


Validation Brier Score: 0.2116502188623727


In [65]:
print("X_test columns:", X_test.columns)
print("X_test shape:", X_test.shape)


X_test columns: Index(['WinRateDifference', 'PointDifferentialDifference'], dtype='object')
X_test shape: (131407, 2)


In [66]:
# --- Assuming test_df was created from SampleSubmissionStage2.csv ---
# Extract Season, Team1ID, and Team2ID from 'ID'
test_df['Season'] = test_df['ID'].str[:4].astype(int)
test_df['Team1ID'] = test_df['ID'].str.split('_').str[1].astype(int)
test_df['Team2ID'] = test_df['ID'].str.split('_').str[2].astype(int)

# Merge team statistics for WinRate and PointDifferential
# For Team1:
test_df = pd.merge(
    test_df,
    team_stats[['TeamID', 'WinRate', 'PointDifferential']],
    left_on='Team1ID',
    right_on='TeamID',
    how='left'
).rename(columns={'WinRate': 'WinRate_Team1', 'PointDifferential': 'PointDifferential_Team1'}).drop(columns=['TeamID'])

# For Team2:
test_df = pd.merge(
    test_df,
    team_stats[['TeamID', 'WinRate', 'PointDifferential']],
    left_on='Team2ID',
    right_on='TeamID',
    how='left'
).rename(columns={'WinRate': 'WinRate_Team2', 'PointDifferential': 'PointDifferential_Team2'}).drop(columns=['TeamID'])

# Merge home court advantage for Team1 and Team2
# For Team1:
test_df = pd.merge(
    test_df,
    home_adv[['TeamID', 'HomeWinRate']],
    left_on='Team1ID',
    right_on='TeamID',
    how='left'
).rename(columns={'HomeWinRate': 'HomeWinRate_Team1'}).drop(columns=['TeamID'])

# For Team2:
test_df = pd.merge(
    test_df,
    home_adv[['TeamID', 'HomeWinRate']],
    left_on='Team2ID',
    right_on='TeamID',
    how='left'
).rename(columns={'HomeWinRate': 'HomeWinRate_Team2'}).drop(columns=['TeamID'])

# Compute feature differences
test_df['WinRateDifference'] = test_df['WinRate_Team1'] - test_df['WinRate_Team2']
test_df['PointDifferentialDifference'] = test_df['PointDifferential_Team1'] - test_df['PointDifferential_Team2']
test_df['HomeWinRateDifference'] = test_df['HomeWinRate_Team1'] - test_df['HomeWinRate_Team2']

# Now, build X_test with all three features:
X_test = test_df[['WinRateDifference', 'PointDifferentialDifference', 'HomeWinRateDifference']]
print("Updated X_test columns:", X_test.columns)
print("Updated X_test shape:", X_test.shape)


NameError: name 'test_df' is not defined

In [67]:
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-66-767124e631a3> in <cell line: 3>()
      1 # --- Assuming test_df was created from SampleSubmissionStage2.csv ---
      2 # Extract Season, Team1ID, and Team2ID from 'ID'
----> 3 test_df['Season'] = test_df['ID'].str[:4].astype(int)
      4 test_df['Team1ID'] = test_df['ID'].str.split('_').str[1].astype(int)
      5 test_df['Team2ID'] = test_df['ID'].str.split('_').str[2].astype(int)

NameError: name 'test_df' is not defined

SyntaxError: invalid decimal literal (<ipython-input-67-2f9a3509df05>, line 3)

In [68]:
import pandas as pd

# Load test_df from SampleSubmissionStage2.csv
test_df = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv")
print("Loaded test_df:")
print(test_df.head())

# Extract Season, Team1ID, and Team2ID from 'ID'
test_df['Season'] = test_df['ID'].str[:4].astype(int)
test_df['Team1ID'] = test_df['ID'].str.split('_').str[1].astype(int)
test_df['Team2ID'] = test_df['ID'].str.split('_').str[2].astype(int)

# Merge team statistics for Team1
test_df = pd.merge(
    test_df,
    team_stats[['TeamID', 'WinRate', 'PointDifferential']],
    left_on='Team1ID',
    right_on='TeamID',
    how='left'
).rename(columns={'WinRate': 'WinRate_Team1', 'PointDifferential': 'PointDifferential_Team1'}).drop(columns=['TeamID'])

# Merge team statistics for Team2
test_df = pd.merge(
    test_df,
    team_stats[['TeamID', 'WinRate', 'PointDifferential']],
    left_on='Team2ID',
    right_on='TeamID',
    how='left'
).rename(columns={'WinRate': 'WinRate_Team2', 'PointDifferential': 'PointDifferential_Team2'}).drop(columns=['TeamID'])

# Merge home court advantage for Team1
test_df = pd.merge(
    test_df,
    home_adv[['TeamID', 'HomeWinRate']],
    left_on='Team1ID',
    right_on='TeamID',
    how='left'
).rename(columns={'HomeWinRate': 'HomeWinRate_Team1'}).drop(columns=['TeamID'])

# Merge home court advantage for Team2
test_df = pd.merge(
    test_df,
    home_adv[['TeamID', 'HomeWinRate']],
    left_on='Team2ID',
    right_on='TeamID',
    how='left'
).rename(columns={'HomeWinRate': 'HomeWinRate_Team2'}).drop(columns=['TeamID'])

# Compute feature differences
test_df['WinRateDifference'] = test_df['WinRate_Team1'] - test_df['WinRate_Team2']
test_df['PointDifferentialDifference'] = test_df['PointDifferential_Team1'] - test_df['PointDifferential_Team2']
test_df['HomeWinRateDifference'] = test_df['HomeWinRate_Team1'] - test_df['HomeWinRate_Team2']

# Create X_test with all three features
X_test = test_df[['WinRateDifference', 'PointDifferentialDifference', 'HomeWinRateDifference']]
print("Updated X_test columns:", X_test.columns)
print("Updated X_test shape:", X_test.shape)



Loaded test_df:
               ID  Pred
0  2025_1101_1102   0.5
1  2025_1101_1103   0.5
2  2025_1101_1104   0.5
3  2025_1101_1105   0.5
4  2025_1101_1106   0.5
Updated X_test columns: Index(['WinRateDifference', 'PointDifferentialDifference',
       'HomeWinRateDifference'],
      dtype='object')
Updated X_test shape: (131407, 3)


In [69]:
test_predictions = lgb_model.predict_proba(X_test)[:, 1]
test_df['Pred'] = test_predictions


In [70]:
print("Loaded test_df:")
print(test_df.head())
print("Updated X_test columns:", X_test.columns)
print("Updated X_test shape:", X_test.shape)


Loaded test_df:
               ID      Pred  Season  Team1ID  Team2ID  WinRate_Team1  \
0  2025_1101_1102  0.244046    2025     1101     1102       0.473354   
1  2025_1101_1103  0.172975    2025     1101     1103       0.473354   
2  2025_1101_1104  0.069555    2025     1101     1104       0.473354   
3  2025_1101_1105  0.664709    2025     1101     1105       0.473354   
4  2025_1101_1106  0.587721    2025     1101     1106       0.473354   

   PointDifferential_Team1  WinRate_Team2  PointDifferential_Team2  \
0                -1.442006       0.428144                -2.013473   
1                -1.442006       0.657264                 4.988717   
2                -1.442006       0.612500                 4.797222   
3                -1.442006       0.317891                -6.690096   
4                -1.442006       0.406627                -3.947289   

   HomeWinRate_Team1  HomeWinRate_Team2  WinRateDifference  \
0           0.509934           0.674825           0.045211   
1     

In [72]:
# Generate predictions for the test set.
# This will produce probabilities for the event that the lower TeamID wins.
test_predictions = lgb_model.predict_proba(X_test)[:, 1]

# Add the predictions to the test_df DataFrame.
test_df['Pred'] = test_predictions

# Prepare the final submission DataFrame with only the required columns (ID and Pred)
final_submission = test_df[['ID', 'Pred']]

# Check the first few rows and the shape of the submission
print(final_submission.head())
print("Final submission shape:", final_submission.shape)

# Save the final submission file as CSV.
final_submission.to_csv("submission.csv", index=False)
print("Final submission file saved as 'submission.csv'")


               ID      Pred
0  2025_1101_1102  0.244046
1  2025_1101_1103  0.172975
2  2025_1101_1104  0.069555
3  2025_1101_1105  0.664709
4  2025_1101_1106  0.587721
Final submission shape: (131407, 2)
Final submission file saved as 'submission.csv'
